In [1]:
import cv2
import mediapipe as mp
import numpy as np
import csv

import os
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
cap = cv2.VideoCapture (0)
height= cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width= cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get (cv2.CAP_PROP_FPS)
videoWriter = cv2.VideoWriter('head_exces.mp4', cv2.VideoWriter_fourcc('P','I', 'M','1'), fps, (int(width), int(height)))
while cap.isOpened():
    ret, frame = cap.read()
    try:
        cv2.imshow('head_exces', frame)
        videoWriter.write(frame)
    except  Exception as e:
        break
    if cv2.waitKey (10) & 0xFF == ord('q'):
        break
cap.release()
videoWriter.release()
cv2.destroyAllWindows()

In [3]:
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [12]:
with open('feature_head_exces.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',',quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [13]:
def export_landmark(results, action):
    try:
        keypoints = [action]  # Initialize the list with the action label

        # Extract pose landmarks and append them to the keypoints list
        for landmark in results.pose_landmarks.landmark:
            keypoints.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])

        # Open the CSV file in 'a' (append) mode and write the keypoints
        with open('feature_head_exces.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(f"Error in export_landmark: {e}")

In [14]:

# Create a VideoCapture object to open the video file
video_path = 'C:/Projects/FinalProject/excesise_proj/head_exces/head_exces.mp4'
cap = cv2.VideoCapture(video_path)

# Check if the video file is opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Initialize a variable to keep track of the current pose
current_pose = None

# Create a dictionary to map keys to pose labels
key_to_pose = {
    ord('w'): 'up',
    ord('s'): 'down',
    ord('a'): 'left',
    ord('d'): 'right',
    ord('e'): 'forword'
}

# Main processing loop
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break  # Exit the loop when there are no more frames

        # Convert frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform pose detection
        results = pose.process(image)

        # Convert back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw landmarks and lines on the frame
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Check for key presses and export landmarks
        key = cv2.waitKey(1)
        if key in key_to_pose:
            current_pose = key_to_pose[key]
            #print(current_pose)
            export_landmark(results, current_pose)

        # Display the frame
        cv2.imshow('webcam feed', image)

        # Check for 'q' key to quit
        if key == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [26]:
df = pd.read_csv('C:/Projects/FinalProject/excesise_proj/head_exces/feature_head_exces.csv')

In [27]:
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,forword,0.546651,0.601508,-0.851775,0.999506,0.576177,0.552339,-0.799037,0.999085,0.591478,...,0.331020,0.000032,0.600760,2.699876,-0.263274,0.000061,0.450077,2.690138,-0.254821,0.000081
1,forword,0.547027,0.602032,-0.858165,0.999468,0.576934,0.552776,-0.808009,0.999036,0.592480,...,0.422921,0.000032,0.596925,2.699874,-0.231794,0.000060,0.449171,2.690914,-0.157405,0.000080
2,forword,0.548140,0.601885,-0.748559,0.999480,0.578087,0.553228,-0.701778,0.999082,0.593837,...,0.288844,0.000033,0.599004,2.700003,-0.206269,0.000064,0.452093,2.688481,-0.292120,0.000087
3,forword,0.547780,0.601288,-0.795835,0.999496,0.578112,0.552947,-0.744792,0.999114,0.593848,...,0.260933,0.000032,0.594922,2.709265,-0.233341,0.000064,0.446413,2.700713,-0.300552,0.000088
4,forword,0.547067,0.587269,-0.803245,0.999505,0.577076,0.542789,-0.749927,0.999128,0.592115,...,0.286784,0.000031,0.594055,2.703555,-0.233464,0.000060,0.438124,2.694698,-0.277678,0.000085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,forword,0.541441,0.643891,-0.915911,0.999723,0.566563,0.595828,-0.879130,0.999342,0.580379,...,0.305493,0.000027,0.577970,2.515942,-0.242121,0.000068,0.445298,2.499604,-0.196127,0.000063
323,forword,0.543542,0.658762,-0.947766,0.999626,0.569738,0.609617,-0.913417,0.999150,0.583848,...,0.088228,0.000030,0.593098,2.516876,-0.377342,0.000070,0.465244,2.500032,-0.416833,0.000067
324,forword,0.553915,0.662154,-1.021262,0.999633,0.579870,0.613635,-0.986208,0.999165,0.594374,...,0.268498,0.000029,0.591916,2.517749,-0.238626,0.000066,0.465686,2.503366,-0.254718,0.000063
325,forword,0.572227,0.664720,-1.019886,0.999579,0.595533,0.615643,-0.982924,0.999083,0.608873,...,0.335132,0.000032,0.596845,2.522515,-0.165719,0.000073,0.467254,2.512269,-0.193205,0.000067


In [28]:
x = df.drop('class', axis=1)
y=df['class']

In [29]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=1234)

In [31]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=5)

model.fit(x_train, y_train)

KNeighborsClassifier()

In [32]:
y_pred = model.predict(x_test)

In [33]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.95
              precision    recall  f1-score   support

        down       1.00      1.00      1.00        10
     forword       0.90      0.97      0.94        38
        left       1.00      0.87      0.93        15
       right       1.00      0.94      0.97        18
          up       0.94      0.94      0.94        18

    accuracy                           0.95        99
   macro avg       0.97      0.95      0.96        99
weighted avg       0.95      0.95      0.95        99

[[10  0  0  0  0]
 [ 0 37  0  0  1]
 [ 0  2 13  0  0]
 [ 0  1  0 17  0]
 [ 0  1  0  0 17]]


In [35]:
import pickle
model_file = 'C:/Projects/FinalProject/excesise_proj/head_exces/head_exces1.pkl'
with open(model_file, 'wb') as file:
    pickle.dump(model, file)
print(f"Model saved as {model_file}")

Model saved as C:/Users/shana/OneDrive/Desktop/excesise_proj/head_exces/head_exces1.pkl


In [34]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

# Initialize MediaPipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Initialize variables
cap = cv2.VideoCapture(0)
state = ''

# Mapping of classes to actions
class_to_action = {
    'forward': 'forward',
    'left': 'left',
    'right': 'right',
    'up': 'up',
    'down': 'down'
}

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        try:
            row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            x = pd.DataFrame([row], columns=landmarks[1:])
            state_class = model.predict(x)[0]
            state_prob = model.predict_proba(x)[0]
            #print(f"State: {state_class}, State Prob: {state_prob}")

            if state_class in class_to_action and state_prob[state_prob.argmax()] >= 0.7:
                state = class_to_action[state_class]

            cv2.rectangle(image, (0,0), (250,60), (245,117,16), -1)

            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            
            cv2.putText(image, state_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            
            cv2.putText(image, str(round(state_prob[np.argmax(state_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
        except Exception as e:
            print(f"Error: {e}")

        cv2.imshow('webcam feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'


In [32]:
import joblib
joblib.dump(model, 'head_exces.pkl')

['head_exces.pkl']

In [33]:
#model = joblib.load('C:/Projects/FinalProject/excesise_proj/head_exces/head_exces.pkl')